In [17]:
import csv
import pandas as pd
import pg8000

# Mouse

In [18]:
mshiny = pd.read_csv('c:\users\lisak\documents\stash\project\SMART_goals\data\mouse_anno_table.csv')

In [19]:
mshiny

,Unnamed: 0,anno.sample_id,anno.batch_id,anno.batch_label,anno.batch_color,anno.cell_name_id,anno.cell_name_label,anno.cell_name_color,anno.spec_id_id,anno.spec_id_label,...,anno.ref_version_color,anno.cre_from_lims,anno.reporter,anno.marker_genes,anno.topLeaf_label,anno.res.index,anno.cluster_detail_id,anno.cluster_detail_color,anno.topLeaf_id,anno.topLeaf_color
0,1,P1S4_160318_069_A01,1,BT014,#FF0000,917,Ndnf-IRES2-dgCre;Ai14-234626.02.L.01,#91FF00,5,515771409,...,#FF0000,Ndnf,NaN,Lamp5 Pdlim5:Nxph1 Tox3 Caln1 Ramp3 Atp6ap1l L...,Lamp5 Pdlim5,1.000000,7.0,#FF7290,9,#FF9300
1,2,P1S4_160407_108_A01,1,BT014,#FF0000,930,Ndnf-IRES2-dgCre;Ai14-237573.07.R.01,#8CFF00,494,541458435,...,#FF0000,Ndnf,NaN,Lamp5 Pdlim5:Nxph1 Ramp3 Atp6ap1l Sv2c Egfr To...,Lamp5 Pdlim5,1.000000,7.0,#FF7290,9,#FF9300
2,3,P1S4_160407_109_A01,1,BT014,#FF0000,926,Ndnf-IRES2-dgCre;Ai14-237573.07.L.01,#8EFF00,3,515770604,...,#FF0000,Ndnf,NaN,Lamp5 Pdlim5:Nxph1 Tox3 Caln1 Luzp2 Sv2c Tox2 ...,Lamp5 Pdlim5,1.000000,7.0,#FF7290,9,#FF9300
3,4,P1S4_160407_112_A01,1,BT014,#FF0000,927,Ndnf-IRES2-dgCre;Ai14-237573.07.L.02,#8DFF00,495,541458440,...,#FF0000,Ndnf,NaN,Lamp5 Pdlim5:Nxph1 Tox3 Caln1 Ramp3 Atp6ap1l L...,Lamp5 Pdlim5,1.000000,7.0,#FF7290,9,#FF9300
4,5,P1S4_160407_113_A01,1,BT014,#FF0000,928,Ndnf-IRES2-dgCre;Ai14-237573.07.L.03,#8DFF00,496,541458444,...,#FF0000,Ndnf,NaN,n43:Sema3e Rxfp1 Ptprt Prox1 Ust Tshz3 Fam19a2...,Lamp5 Egln3_3,0.802691,2.0,#808080,6,#FF5C00
5,6,P1S4_160407_114_A01,1,BT014,#FF0000,929,Ndnf-IRES2-dgCre;Ai14-237573.07.L.04,#8CFF00,497,541458448,...,#FF0000,Ndnf,NaN,Lamp5 Egln3_3:Hs3st4 Drd1 Baiap2 Rgs4 Pcdh11x ...,Lamp5 Egln3_3,1.000000,6.0,#DD6091,6,#FF5C00
6,7,P1S4_160408_121_A01,1,BT014,#FF0000,3308,Vip-IRES-Cre;Ai14-240611.02.02.01,#FF00D5,538,541474914,...,#FF0000,Vip,NaN,Vip Gpc3:Npy2r Nnat Cpne4 Id3 Penk Gpc3 Lin7a ...,Vip Gpc3,1.000000,24.0,#B09FFF,46,#00BEFF
7,8,P1S4_160408_123_A01,1,BT014,#FF0000,3309,Vip-IRES-Cre;Ai14-240611.02.02.02,#FF00D4,539,541474930,...,#FF0000,Vip,NaN,Vip Cbln4_2:Kirrel3 Mgat4c Penk Stac2 Ppapdc1a...,Vip Cbln4_2,1.000000,29.0,#FF00B3,43,#00F6FF
8,9,P1S4_160408_124_A01,1,BT014,#FF0000,3303,Vip-IRES-Cre;Ai14-240611.01.01.01,#FF00D7,532,541474046,...,#FF0000,Vip,NaN,Vip Cbln4_2:Kirrel3 Igfbp4 Cbln4 Mgat4c Penk P...,Vip Cbln4_2,1.000000,29.0,#FF00B3,43,#00F6FF
9,10,P1S4_160408_125_A01,1,BT014,#FF0000,3304,Vip-IRES-Cre;Ai14-240611.01.01.02,#FF00D6,533,541474088,...,#FF0000,Vip,NaN,n38:Adarb2 Nfib Nfix Prox1 Cacna2d1 Sorcs3 Cry...,Vip Crispld2_1,0.562351,121.0,#808080,44,#00E3FF


In [20]:
def _connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    conn = pg8000.connect(user=user, host=host, database=database, password=password, port=port)
    return conn, conn.cursor()

def _select(cursor, query):
    cursor.execute(query)
    columns = [ d[0] for d in cursor.description ]
    return [ dict(zip(columns, c)) for c in cursor.fetchall() ]

def limsquery(query, user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    """A function that takes a string containing a SQL query, connects to the LIMS database and outputs the result."""
    conn, cursor = _connect(user, host, database, password, port)
    try:
        results = _select(cursor, query)
    finally:
        
        #THESE ARE IMPORTANT!!!!!!
        #Every query needs to be closed when done
        cursor.close()             
        conn.close()
    return results


#this last function will take our query results and put them in a dataframe so that they are easy to work with
def get_lims_dataframe(query):
    '''Return a dataframe with lims query'''
    result = limsquery(query)
    try:
        data_df = pd.DataFrame(data=result, columns=result[0].keys())
    except IndexError:
        print "Could not find results for your query."
        data_df = pd.DataFrame()
    return data_df

In [31]:
imgs_query = "Select distinct cell.id, cell.name, array_to_string(array_agg(DISTINCT tag.name), '_AND_') \
FROM specimens cell JOIN ephys_roi_results err on err.id = cell.ephys_roi_result_id \
JOIN specimen_tags_specimens sptagsp on sptagsp.specimen_id = cell.id \
JOIN specimen_tags tag on tag.id = sptagsp.specimen_tag_id and tag.id in (602120185, 602122082) \
GROUP BY cell.id \
ORDER BY 1"
imgs_df = get_lims_dataframe(imgs_query)
imgs_df

,array_to_string,id,name
0,63x no go,479092788,Scnn1a-Tg3-Cre;Ai14-195195.05.02.01
1,63x no go,486748229,H15.03.005.01.14.01
2,63x no go,488402499,H15.06.017.03.09.01
3,63x go,505689242,H16.06.003.01.03.03
4,63x no go,508302595,H16.06.004.01.03.03
5,63x no go,508366192,H16.06.004.01.05.01
6,63x go,508376177,H16.06.004.01.11.02
7,63x no go,508381568,H16.06.004.02.04.03
8,63x go,508390609,H16.06.004.01.11.04
9,63x go,508400454,H16.06.004.02.13.01


In [21]:
lims_query = "SELECT specimens.name, specimens.patched_cell_container, err.recording_date, specimens.ephys_roi_result_id, err.id \
            FROM ephys_roi_results err JOIN specimens ON specimens.ephys_roi_result_id = err.id \
            WHERE patched_cell_container != 'None'"
lims_df = get_lims_dataframe(lims_query)
lims_df.head()

,ephys_roi_result_id,patched_cell_container,name,recording_date,id
0,517754056,P1S4_160510_455_A01,Rorb-IRES2-Cre-D;Ai14-245133.07.01.01,2016-05-10 10:48:36,517754056
1,517980977,P1S4_160511_471_A01,Scnn1a-Tg2-Cre;Ai14-245345.02.02.01,2016-05-11 11:38:48,517980977
2,517983617,P1S4_160511_472_A01,Scnn1a-Tg2-Cre;Ai14-245345.07.01.01,2016-05-11 13:07:14,517983617
3,517984206,P1S4_160511_473_A01,Scnn1a-Tg2-Cre;Ai14-245345.07.01.02,2016-05-11 13:48:20,517984206
4,518056970,P1S4_160512_485_A01,Rorb-IRES2-Cre-D;Ai14-245132.07.02.04,2016-05-12 14:24:57,518056970


NOTE: A few recordings in LIMS are not updated - have no ephys sweeps, therefore no recording_date

In [35]:
lims = pd.merge(left = lims_df, right = imgs_df, left_on = 'name', right_on = 'name', how = 'left')
lims

,ephys_roi_result_id,patched_cell_container,name,recording_date,id_x,array_to_string,id_y
0,517754056,P1S4_160510_455_A01,Rorb-IRES2-Cre-D;Ai14-245133.07.01.01,2016-05-10 10:48:36.000,517754056,NaN,NaN
1,517980977,P1S4_160511_471_A01,Scnn1a-Tg2-Cre;Ai14-245345.02.02.01,2016-05-11 11:38:48.000,517980977,NaN,NaN
2,517983617,P1S4_160511_472_A01,Scnn1a-Tg2-Cre;Ai14-245345.07.01.01,2016-05-11 13:07:14.000,517983617,NaN,NaN
3,517984206,P1S4_160511_473_A01,Scnn1a-Tg2-Cre;Ai14-245345.07.01.02,2016-05-11 13:48:20.000,517984206,NaN,NaN
4,518056970,P1S4_160512_485_A01,Rorb-IRES2-Cre-D;Ai14-245132.07.02.04,2016-05-12 14:24:57.000,518056970,NaN,NaN
5,518063401,P1S4_160512_487_A01,Rorb-IRES2-Cre-D;Ai14-245132.08.01.01,2016-05-12 15:02:14.000,518063401,NaN,NaN
6,518233727,P1S4_160513_491_A01,Scnn1a-Tg2-Cre;Ai14-245344.06.01.01,2016-05-13 10:57:12.000,518233727,NaN,NaN
7,518296209,P1S4_160513_493_A01,Scnn1a-Tg2-Cre;Ai14-245344.03.01.02,2016-05-13 13:15:19.000,518296209,NaN,NaN
8,518318886,P1S4_160513_494_A01,Scnn1a-Tg2-Cre;Ai14-245344.03.01.03,2016-05-13 14:02:17.000,518318886,NaN,NaN
9,518625250,P1S4_160516_001_A01,Chat-IRES-Cre;Ai14-245957.06.01.01,2016-05-16 11:24:01.000,518625250,NaN,NaN


In [37]:
smart_data = pd.merge(left = mshiny, right = lims, left_on = 'anno.sample_id', right_on = 'patched_cell_container', how = 'outer')
smart_data

,Unnamed: 0,anno.sample_id,anno.batch_id,anno.batch_label,anno.batch_color,anno.cell_name_id,anno.cell_name_label,anno.cell_name_color,anno.spec_id_id,anno.spec_id_label,...,anno.cluster_detail_color,anno.topLeaf_id,anno.topLeaf_color,ephys_roi_result_id,patched_cell_container,name,recording_date,id_x,array_to_string,id_y
0,1.0,P1S4_160318_069_A01,1.0,BT014,#FF0000,917.0,Ndnf-IRES2-dgCre;Ai14-234626.02.L.01,#91FF00,5.0,515771409.0,...,#FF7290,9.0,#FF9300,NaN,NaN,NaN,NaT,NaN,NaN,NaN
1,2.0,P1S4_160407_108_A01,1.0,BT014,#FF0000,930.0,Ndnf-IRES2-dgCre;Ai14-237573.07.R.01,#8CFF00,494.0,541458435.0,...,#FF7290,9.0,#FF9300,NaN,NaN,NaN,NaT,NaN,NaN,NaN
2,3.0,P1S4_160407_109_A01,1.0,BT014,#FF0000,926.0,Ndnf-IRES2-dgCre;Ai14-237573.07.L.01,#8EFF00,3.0,515770604.0,...,#FF7290,9.0,#FF9300,NaN,NaN,NaN,NaT,NaN,NaN,NaN
3,4.0,P1S4_160407_112_A01,1.0,BT014,#FF0000,927.0,Ndnf-IRES2-dgCre;Ai14-237573.07.L.02,#8DFF00,495.0,541458440.0,...,#FF7290,9.0,#FF9300,NaN,NaN,NaN,NaT,NaN,NaN,NaN
4,5.0,P1S4_160407_113_A01,1.0,BT014,#FF0000,928.0,Ndnf-IRES2-dgCre;Ai14-237573.07.L.03,#8DFF00,496.0,541458444.0,...,#808080,6.0,#FF5C00,NaN,NaN,NaN,NaT,NaN,NaN,NaN
5,6.0,P1S4_160407_114_A01,1.0,BT014,#FF0000,929.0,Ndnf-IRES2-dgCre;Ai14-237573.07.L.04,#8CFF00,497.0,541458448.0,...,#DD6091,6.0,#FF5C00,NaN,NaN,NaN,NaT,NaN,NaN,NaN
6,7.0,P1S4_160408_121_A01,1.0,BT014,#FF0000,3308.0,Vip-IRES-Cre;Ai14-240611.02.02.01,#FF00D5,538.0,541474914.0,...,#B09FFF,46.0,#00BEFF,NaN,NaN,NaN,NaT,NaN,NaN,NaN
7,8.0,P1S4_160408_123_A01,1.0,BT014,#FF0000,3309.0,Vip-IRES-Cre;Ai14-240611.02.02.02,#FF00D4,539.0,541474930.0,...,#FF00B3,43.0,#00F6FF,NaN,NaN,NaN,NaT,NaN,NaN,NaN
8,9.0,P1S4_160408_124_A01,1.0,BT014,#FF0000,3303.0,Vip-IRES-Cre;Ai14-240611.01.01.01,#FF00D7,532.0,541474046.0,...,#FF00B3,43.0,#00F6FF,NaN,NaN,NaN,NaT,NaN,NaN,NaN
9,10.0,P1S4_160408_125_A01,1.0,BT014,#FF0000,3304.0,Vip-IRES-Cre;Ai14-240611.01.01.02,#FF00D6,533.0,541474088.0,...,#808080,44.0,#00E3FF,NaN,NaN,NaN,NaT,NaN,NaN,NaN
